# Despacho múltiplo

Neste notebook, vamos explorar o conceito de [despacho múltiplo](https://en.wikipedia.org/wiki/Multiple_dispatch), que é um recurso chave da linguagem Julia.

Para entender despacho múltiplo em Julia, vamos começar com um exemplo que nós já vimos.

Em Julia, podemos declarar funções sem informar o tipo dos argumentos da função.
Por exemplo:

In [1]:
f(x) = x^2

f (generic function with 1 method)

Então a Julia vai determinar automaticamente sobre quais tipos de argumentos faz sentido aplicar a função, e sobre quais tipos não faz:

In [2]:
f(9)

81

In [3]:
f([1, 2, 3])

MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:885
  ^(!Matched::Regex, ::Integer) at regex.jl:712
  ^(!Matched::Missing, ::Integer) at missing.jl:155
  ...

## Especificando os tipos dos argumentos

Temos a opção de declarar explicitamente quais tipos de argumentos uma função pode receber.
Por exemplo, vamos definir uma função `teste` que só pode receber argumentos do tipo `String`:

In [4]:
teste(x::String, y::String) = println("Os argumentos x e y são strings")

teste (generic function with 1 method)

Vamos verificar que a função `teste` age sobre strings, mas não age sobre argumentos de outros tipos:

In [5]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


In [6]:
teste(2, 3)

MethodError: MethodError: no method matching teste(::Int64, ::Int64)

Para fazer com que a função `teste` aceite números inteiros, vamos definir `teste` para argumentos do tipo `Int`:

In [7]:
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros")

teste (generic function with 2 methods)

In [8]:
teste(2, 3)

Os argumentos x e y são números inteiros


Agora `teste` age sobre os inteiros.
Mas observe, a função `teste` continua definida quando `x` e `y` são strings:

In [9]:
teste("despacho", "múltiplo")

Os argumentos x e y são strings


Quando declaramos
```julia
teste(x::Int, y::Int) = println("Os argumentos x e y são números inteiros"),
```
não removemos ou substituímos
```julia
teste(x::String, y::String),
```
simplesmente adicionamos um método à função genérica `teste`.

Uma **função genérica** é um conceito abstrato associado a uma operação.
Por exemplo, a função genérica `+` representa o conceito de adição.
Um **método** é uma implementação específica de uma função genérica para um tipo específico de argumento.
Por exemplo, a função `+` possui métodos para números de precisão flutuante, inteiros, matrizes, etc.
Podemos usar `methods` para ver os métodos que a função `teste` possui:

In [10]:
methods(teste)

# 2 methods for generic function "teste":
[1] teste(x::Int64, y::Int64) in Main at In[7]:1
[2] teste(x::String, y::String) in Main at In[4]:1

E podemos ver os métodos da função `+`:

In [11]:
methods(+)

# 166 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:286
[2] +(x::Bool, y::Bool) in Base at bool.jl:96
[3] +(x::Bool) in Base at bool.jl:93
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:104
[5] +(x::Bool, z::Complex) in Base at complex.jl:293
[6] +(a::Float16, b::Float16) in Base at float.jl:398
[7] +(x::Float32, y::Float32) in Base at float.jl:400
[8] +(x::Float64, y::Float64) in Base at float.jl:401
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:287
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:301
[11] +(::Missing, ::Missing) in Base at missing.jl:114
[12] +(::Missing) in Base at missing.jl:100
[13] +(::Missing, ::Number) in Base at missing.jl:115
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:107
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:414
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:515
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:514
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:513
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:473
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:521
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:527
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:564
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:557
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:551
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:410
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:379
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:386
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:394
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:402
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/periods.jl:341
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/periods.jl:339
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/periods.jl:369
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /usr/share/julia/stdlib/v1.4/Dates/src/arithmetic.jl:76
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /usr/share/julia/stdlib/v1.4/LinearAlgebra/src/uniformscaling.jl:115
[44] +(a::Pkg.Resolve.VersionWeight, b::Pkg.Resolve.VersionWeight) in Pkg.Resolve at /usr/share/julia/stdlib/v1.4/Pkg/src/Resolve/versionweights.jl:22
[45] +(a::Pkg.Resolve.FieldValue, b::Pkg.Resolve.FieldValue) in Pkg.Resolve at /usr/share/julia/stdlib/v1.4/Pkg/src/Resolve/fieldvalues.jl:43
[46] +(y::AbstractFloat, x::Bool) in Base at bool.jl:106
[47] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32,

Portanto, agora podemos aplicar `teste` a strings e inteiros.
Quando aplicamos `teste` a um par de argumentos, a Julia vai inferir os tipos dos argumentos e despachar o método apropriado.
Isso é **despacho múltiplo**.
Despacho múltiplo torna os programas genéricos e rápidos.

Para ver qual método está sendo despachado quando chamamos uma função genérica, usamos a macro `@which`:

In [12]:
@which teste(2, 3)

teste(x::Int64, y::Int64) in Main at In[7]:1

In [13]:
@which 3.0 + 5.0

+(x::Float64, y::Float64) in Base at float.jl:401